In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
import yfinance as yf
import datetime
from IPython.display import clear_output
import pytz
import timeit
#import plotly.io as pio
#pio.renderers.default = "browser"

In [2]:
def resample_data(df, interval):
    # Define the resampling logic
    ohlcv_dict = {
         'Open': 'first',
         'High': 'max',
         'Low': 'min',
         'Close': 'last',
         'Volume': 'sum'
    }
    # 10T for 10 minutes, D for 1 day and M for 1 month
    # Resample the data
    df = df.resample(interval).agg(ohlcv_dict)
    df = df.dropna()
    return df

In [3]:
from pyti.exponential_moving_average import exponential_moving_average as ema
import pandas_ta as ta

# Calculate indicators EMA9, EMA13, VWAP
def vwap(df):
    v = df.Volume.values
    tp = (df.High+df.Low+df.Close).div(3).values
    df['vwap'] = (tp*v).cumsum() / v.cumsum()
    return df

def calculate_indicators(df):
    df = df.groupby(df.index.date, group_keys=False).apply(vwap)
    df['ema_nine'] = ta.ema(df['Close'], length=9)
    df['ema_thirteen'] = ta.ema(df['Close'], length=13)
    #df['ema_nine'] = ema(df['Close'], 9)
    #df['ema_thirteen'] = ema(df['Close'], 13)
    return df
    
def pull_intraday_data(ticker_symbol, today_date_str):
    intraday_data = yf.download(tickers=ticker_symbol, period='2d', interval='1m', auto_adjust=True)
    intraday_data = resample_data(intraday_data, '3T')
    intraday_data = calculate_indicators(intraday_data)
    
    
    return intraday_data.loc[today_date_str]

# Initialize intraday strategy columns
def initialize_columns(intraday_df):
    # Create columns for Triple EMA
    intraday_df['triple_ema_low_closeness'] = 0
    intraday_df['triple_ema_high_closeness'] = 0
    intraday_df['triple_ema_hold_rule_one'] = 0
    intraday_df['triple_ema_retest_rule_two'] = 0
    intraday_df['triple_ema_hold_above_signal'] = 0
    intraday_df['triple_ema_hold_above_signal_cummulative'] = 0
    intraday_df['triple_ema_hold_below_signal'] = 0
    intraday_df['triple_ema_hold_below_signal_cummulative'] = 0 

    # Create columns for EMA 9 and VWAP crossover
    intraday_df['crossover_signal'] = 0
    intraday_df['crossover_position'] = 0

    # Create columns for vwap_retest
    intraday_df['vwap_retest_signal_rule_one'] = 0
    intraday_df['vwap_confirmation_candle_signal_rule_two'] = False
    intraday_df['candle_open_close_midpoint'] = 0
    intraday_df['current_vwap_local_max'] = 0
    intraday_df['prev_vwap_local_max'] = 0
    intraday_df['current_vwap_local_min'] = np.inf
    intraday_df['prev_vwap_local_min'] = np.inf
    
    return intraday_df
    


In [4]:
import plotly.graph_objs as go

def create_graph():
    f = go.FigureWidget()
    f.add_candlestick()
    f.add_scatter()
    f.add_scatter()
    f.add_scatter()
    f.update_xaxes(
            rangeslider_visible=True,
            rangebreaks=[
                # NOTE: Below values are bound (not single values), ie. hide x to y
                dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
                dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
                # dict(values=["2020-12-25", "2021-01-01"])  # hide holidays (Christmas and New Year's, etc)
            ]
        )
    display(f)
    return f
def update_graph(df, f):
    # Update candlestick
    candlestick = f.data[0]
    candlestick.x = x=df.index
    candlestick.open = df['Open']
    candlestick.high = df['High']
    candlestick.low = df['Low']
    candlestick.close = df['Close']
    candlestick.name = 'market data'
    
    #Update VWAP
    scatter_vwap = f.data[1]
    scatter_vwap.x = df.index
    scatter_vwap.y = df['vwap']
    scatter_vwap.mode='lines'
    scatter_vwap.name = 'VWAP'
    scatter_vwap.line = dict(color='teal')
    
    #Update EMA9
    scatter_ema9 = f.data[2]
    scatter_ema9.x = df.index
    scatter_ema9.y = df['ema_nine']
    scatter_ema9.mode='lines'
    scatter_ema9.name = 'EMA9'
    scatter_ema9.line = dict(color='blue')
    
    #Update EMA13
    scatter_ema13 = f.data[3]
    scatter_ema13.x = df.index
    scatter_ema13.y = df['ema_thirteen']
    scatter_ema13.mode='lines'
    scatter_ema13.name = 'EMA13'
    scatter_ema13.line = dict(color='purple')
    
    
    return f
def plot_graph(df):
    #clear_output()
    fig = go.Figure()

    fig.add_trace(go.Candlestick(x=df.index,
                                open=df['Open'],
                                high=df['High'],
                                low=df['Low'],
                                close=df['Close'], name = 'market data'))
    fig.add_trace(go.Scatter(x=df.index, y=df['vwap'], mode='lines', name = 'VWAP', 
                  line=dict(color='teal')))
    fig.add_trace(go.Scatter(x=df.index, y=df['ema_nine'], mode='lines', name = 'EMA9', 
                  line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=df.index, y=df['ema_thirteen'], mode='lines', name = 'EMA13', 
                  line=dict(color='purple')))

    #fig.update_layout(xaxis_rangeslider_visible=False)
    fig.update_xaxes(
            rangeslider_visible=True,
            rangebreaks=[
                # NOTE: Below values are bound (not single values), ie. hide x to y
                dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
                dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
                # dict(values=["2020-12-25", "2021-01-01"])  # hide holidays (Christmas and New Year's, etc)
            ]
        )
    display(fig)

In [5]:
# EMA9 + VWAP Combo
# def generate_ema9_vwap_signal(df):
#     # Looking for Bearish Crossover
#     df['bearish_position'] = df['vwap'] > df['ema_nine']
#     df['bearish_pre_position'] = df['bearish_position'].shift(1)
#     #df.dropna(inplace=True) # dropping the NaN values
#     df['bearish_crossover'] = np.where((df['bearish_position'] - df['bearish_pre_position'])==1, -1, 0)
#     # Looking for Bullish Crossover
#     df['bullish_position'] = df['ema_nine'] < df['vwap']
#     df['bullish_pre_position'] = df['bullish_position'].shift(1)
#     #df.dropna(inplace=True) # dropping the NaN values
#     df['bullish_crossover'] = np.where((df['bullish_position'] - df['bullish_pre_position'])==-1, 1, 0)
#     df['crossover'] = df["bearish_crossover"] + df["bullish_crossover"]
#     return df

def generate_ema9_vwap_signal(df):
    # Looking for EMA9 and VWAP Crossover
    #df_temp = df.copy()
    #df_temp['crossover_signal'] = 0
    print("Running Analysis - EMA9 + VWAP Crossover Setup")
    
    df['crossover_signal'] = np.where(df['ema_nine'] > df['vwap'], 1, 0)
    df['crossover_position'] = df['crossover_signal'].diff()    
    return df

def hold_above_or_below(df):
    
    close_to_ema_tolerance = 0.0005 # 0.05%
    
    # Check if last 5 candles are within tolerance    
    df['triple_ema_low_closeness'] = abs(df['Low'] - df['ema_nine']) / df['ema_nine']
    df['triple_ema_hold_above_signal'] = np.where((df['Low'] >= df['ema_nine']) | (df['triple_ema_low_closeness'] <= close_to_ema_tolerance) , 1, 0)
    #df['triple_ema_hold_above_signal'] = np.where(df['Low'] >= df['ema_nine'], 1, 0)
    df['triple_ema_hold_above_signal_cummulative'] = df['triple_ema_hold_above_signal'].cumsum()
    if (df.loc[df.index[-1], 'triple_ema_hold_above_signal_cummulative'] == 5):
        df.loc[df.index[-1], 'triple_ema_hold_rule_one'] = 1
        
        
    # Check if rule 1 fulfilled - 5 Candles below EMA 20
    df['triple_ema_high_closeness'] = abs(df['High'] - df['ema_nine']) / df['ema_nine']
    df['triple_ema_hold_below_signal'] = np.where((df['High'] <= df['ema_nine']) | (df['triple_ema_high_closeness'] <= close_to_ema_tolerance), -1, 0)
    #df['triple_ema_hold_below_signal'] = np.where(df['High'] <= df['ema_nine'], -1, 0)

    df['triple_ema_hold_below_signal_cummulative'] = df['triple_ema_hold_below_signal'].cumsum()
    if (df.loc[df.index[-1], 'triple_ema_hold_below_signal_cummulative'] == -5):
        df.loc[df.index[-1], 'triple_ema_hold_rule_one'] = -1
    
    tolerance  = 0.0005 # 0.05%
    # Check for Long or Short
    if df.loc[df.index[-1], 'triple_ema_hold_rule_one'] == -1:
    
        # Check if rule 2 fulfilled - candle 0 touches/nears EMA20, 0.05% is considered near
        if 0 <= (abs(df.loc[df.index[-1], 'ema_nine'] - df.loc[df.index[-1], 'High'])
                 /df.loc[df.index[-1], 'ema_nine']) <= tolerance:
            df.loc[df.index[-1], 'triple_ema_retest_rule_two'] = -1 
                
    
    elif df.loc[df.index[-1], 'triple_ema_hold_rule_one'] == 1:
        
        # Check if rule 2 fulfilled - candle 0 touches/nears EMA20, 0.05% is considered near
        if 0 <= (abs(df.loc[df.index[-1], 'ema_nine'] - df.loc[df.index[-1], 'Low'])
                 /df.loc[df.index[-1], 'ema_nine']) <= tolerance:
            df.loc[df.index[-1], 'triple_ema_retest_rule_two'] = 1
    
    return df
    
def triple_ema(df):
    #df_temp = df.copy()
    #df_temp['hold_rule_one'] = 0
    #df_temp['retest_rule_two'] = 0
    
    if(df.shape[0] >= 5):
        print("Running Analysis - TripleEMA Setup")
        # Check if last 5 candles are above EMA9
        df_last_5 = df.iloc[-5:]
        df_last_5 = hold_above_or_below(df_last_5)
        # Update last row in main df
        df.iloc[-1] = df_last_5.iloc[-1]

    return df
    
    
def vwap_retest(df):
    print("Running Analysis - VWAP Retest Setup")
    # Check for vwap retest
    tolerance  = 0.002 # 0.05%
    candlewick_midpoint = (df.loc[df.index[-1], 'Open'] + df.loc[df.index[-1], 'Close'])/2
    df.loc[df.index[-1], 'candle_open_close_midpoint'] = candlewick_midpoint
    #candle_midpoint = abs(df.loc[df.index[-1], 'Close'] + df.loc[df.index[-1], 'Open'])/2
    
    # Update Local Max and Min value
    if df.loc[df.index[-1], 'High'] > df.loc[df.index[-1], 'current_vwap_local_max']:
        df.loc[df.index[-1], 'current_vwap_local_max'] = df.loc[df.index[-1], 'High']
        
    if df.loc[df.index[-1], 'Low'] < df.loc[df.index[-1], 'current_vwap_local_min']:
        df.loc[df.index[-1], 'current_vwap_local_min'] = df.loc[df.index[-1], 'Low']
        
    if df.shape[0] > 4 :
        # Determine the last 5 candles, whether they are above or below vwap
        df_last_5 = df.iloc[-5:]
        no_last_5_candle_pos_above = df_last_5[df_last_5['candle_open_close_midpoint'] >= df_last_5['vwap']].shape[0]
        no_last_5_candle_pos_below = df_last_5[df_last_5['candle_open_close_midpoint'] <= df_last_5['vwap']].shape[0]
        
        # Determine the last two candles' price action
        price_action_direction  = df.loc[df.index[-1], 'Close'] - df.loc[df.index[-2], 'Close']
        
        # When candle is above vwap and price action is still decreasing, it's in the direction to retest vwap
        if no_last_5_candle_pos_above >= 3 and  price_action_direction < 0:
            #print("Enter LONG VWAP Retest")
            # Check if the candle's close / low is near vwap
            if (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) or (0 <= (abs(df.loc[df.index[-1], 'Low'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance):
                    df.loc[df.index[-1], 'vwap_retest_signal_rule_one'] = 1
                    
        # When candle is below vwap and price action is still increasing, it's in the direction to retest vwap
        #candlewick_midpoint <= df.loc[df.index[-1], 'vwap']
        elif no_last_5_candle_pos_below >= 3 and  price_action_direction > 0:
            # Check if candle's close / high is near vwap
            if (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) or (0 <= (abs(df.loc[df.index[-1], 'High'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance):
                    df.loc[df.index[-1], 'vwap_retest_signal_rule_one'] = -1
        
        # Only if vwap retest rule 1 is fulfilled, check for confirmation
        if df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] != 0:
            # For Long setup, check if latest candle's close is above the previous candle's close and it closes above vwap
            if (df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] == 1) and (df.loc[df.index[-1], 'Close'] > df.loc[df.index[-2], 'Close']) and (df.loc[df.index[-1], 'Close'] > df.loc[df.index[-1], 'vwap']):
                    df.loc[df.index[-1], 'vwap_confirmation_candle_signal_rule_two'] = True
            # For Short setup, check if confilatestrmation candle's close is below the previous candle's close and it closes below vwap
            elif (df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] == -1) and (df.loc[df.index[-1], 'Close'] < df.loc[df.index[-2], 'Close']) and (df.loc[df.index[-1], 'Close'] < df.loc[df.index[-1], 'vwap']):
                    df.loc[df.index[-1], 'vwap_confirmation_candle_signal_rule_two'] = True
            
#         # Check if previous candle has fulfilled vwap_test
#         if df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] == 0:
#             # Check if latest candle is above or below vwap (using midpoint between HIGH and LOW to decide)
#             # and the price action is on a downtrend
#             # Candle above vwap (Long)
#             if candlewick_midpoint > df.loc[df.index[-1], 'vwap'] :
#                 # Check if candle's low is near vwap
#                 #if (0 <= (abs(df.loc[df.index[-1], 'Low'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance)  or (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) or (0 <= (abs(df.loc[df.index[-1], 'Open'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) \
#                 if (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) \
#                 and price_action_direction < 0:
#                     df.loc[df.index[-1], 'vwap_retest_signal_rule_one'] = 1

#             # Candle below vwap (Short) and price action is on a uptrend
#             elif candlewick_midpoint < df.loc[df.index[-1], 'vwap'] :
#                 # Check if candle's high is near vwap
#                 #if (0 <= (abs(df.loc[df.index[-1], 'High'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) or (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) or (0 <= (abs(df.loc[df.index[-1], 'Open'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) \
#                 if (0 <= (abs(df.loc[df.index[-1], 'Close'] - df.loc[df.index[-1], 'vwap']) /df.loc[df.index[-1], 'vwap']) <= tolerance) \
#                 and price_action_direction > 0:
#                     df.loc[df.index[-1], 'vwap_retest_signal_rule_one'] = -1

#         else:
#             # For Long setup, check if latest candle's close is above the previous candle's close
#             if (df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] == 1) and (df.loc[df.index[-1], 'Close'] > df.loc[df.index[-2], 'Close']):
#                     df.loc[df.index[-1], 'vwap_confirmation_candle_signal_rule_two'] = True
#             # For Short setup, check if confilatestrmation candle's close is below the previous candle's close
#             elif (df.loc[df.index[-2], 'vwap_retest_signal_rule_one'] == -1) and (df.loc[df.index[-1], 'Close'] < df.loc[df.index[-2], 'Close']):
#                     df.loc[df.index[-1], 'vwap_confirmation_candle_signal_rule_two'] = True
    
    # Append to result dataframe
    #result_df = result_df.append(df.iloc[-1])
    return df    

In [ ]:
# Retrieve ticker's symbol from watchlist
watchlist_filename = "./Watchlist_test.csv"
df_watchlist = pd.read_csv(watchlist_filename)  
print(df_watchlist)

# Create a dictionary 
watchlist_dict = pd.Series(None,df_watchlist.Symbol.values).to_dict()
# Initialize empty dataframe
for key, df in watchlist_dict.items():
    watchlist_dict[key] = pd.DataFrame()
    
# # Simulate Live Data
# intraday_data = yf.download(tickers='MSFT', period='4d', interval='1m', auto_adjust=True)
# #intraday_data.to_csv('SE_14012022_VWAP_RETEST_SHORT.csv')
# #intraday_data.index= intraday_data.index.tz_localize(None)
# intraday_data = resample_data(intraday_data, '3T')
# intraday_data = calculate_indicators(intraday_data)

# today_date = datetime.date(2022, 1, 19) # to be replaced by datetime.now()
# today_date_str = today_date.strftime("%Y-%m-%d")
# print(intraday_data.shape)
# # Narrow to today's
# intraday_df_today = intraday_data.loc[today_date_str]

today_date = datetime.date(2022, 2, 2) # to be replaced by datetime.now()
today_date_str = today_date.strftime("%Y-%m-%d")
    
index = 0
interval = 3
max_length = 130 # 3mins chart max length per day for 3mins chart
f = create_graph()
while (index < max_length):
    start_time = timeit.default_timer()
    # Narrow to current index
    datetime_time = datetime.datetime.combine(today_date, datetime.time(9, 30, 0)) + datetime.timedelta(minutes=index*interval) 
        
    # Iterate through watchlist tickers
    for key, df in watchlist_dict.items():
        print(datetime_time, "- Running Analysis on Ticker Symbol: ", key)
        # Pull Live Data
        intraday_df_today = pull_intraday_data(key, today_date_str)
        intraday_df_today = initialize_columns(intraday_df_today)
        
        result_df = watchlist_dict[key]
        time_list = []
        triple_ema_time_list = []
        ema9_crossover_time_list = []
        vwap_time_list = []
        
        # Narrow to current index
        #datetime_time = datetime.datetime.combine(today_date, datetime.time(9, 30, 0)) + datetime.timedelta(minutes=index*interval) 
        #eastern_tzinfo = pytz.timezone("US/Eastern")
        #datetime_time = eastern_tzinfo.localize(datetime.datetime.combine(today_date, datetime.time(9, 30, 0)) + datetime.timedelta(minutes=index*interval), is_dst=None)
        # Convert datetime to string
        #datetime_time_str = datetime_time.strftime('%Y-%m-%d %H:%M:%S%z')
        #print(datetime_time)                                        
        intraday_df = intraday_df_today.loc[:datetime_time]

        # Append latest row to result_df for processing
        result_df = result_df.append(intraday_df.iloc[-1])
        result_df['vwap_confirmation_candle_signal_rule_two'] = result_df['vwap_confirmation_candle_signal_rule_two'].astype('bool')

        # Triple EMA
        temp_start_time = timeit.default_timer()
        result_df = triple_ema(result_df)
        triple_ema_time_list.append(datetime.timedelta(seconds = (timeit.default_timer() - temp_start_time)).total_seconds())

        # EMA9 and VWAP crossover
        temp_start_time = timeit.default_timer()
        result_df = generate_ema9_vwap_signal(result_df)
        ema9_crossover_time_list.append(datetime.timedelta(seconds = (timeit.default_timer() - temp_start_time)).total_seconds())

        # Test for VWAP retest
        temp_start_time = timeit.default_timer()
        result_df = vwap_retest(result_df)
        vwap_time_list.append(datetime.timedelta(seconds = (timeit.default_timer() - temp_start_time)).total_seconds())
        #print(result_df.tail(5))
        #display(result_df.loc[result_df['vwap_retest_signal_rule_one'] == 1].head(10))
        #display(result_df.loc[result_df['vwap_confirmation_candle_signal_rule_two'] == True].head(10))

        time_list.append(datetime.timedelta(seconds = (timeit.default_timer() - start_time)).total_seconds())
        # Plot Graph
        #f = update_graph(intraday_df, f)
        
        watchlist_dict[key] = result_df
        
    #print(index)
    index+=1
    


  Exchange Symbol
0   NASDAQ   MSFT
1   NASDAQ   AAPL
2   NASDAQ   TSLA


<ipython-input-6-04b7f7b508db>:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  watchlist_dict = pd.Series(None,df_watchlist.Symbol.values).to_dict()


FigureWidget({
    'data': [{'type': 'candlestick', 'uid': '6011ab30-8516-46b6-9bca-b2456fe38c04'},
          …

2022-02-02 09:30:00 - Running Analysis on Ticker Symbol:  MSFT
[*********************100%***********************]  1 of 1 completed
Running Analysis - EMA9 + VWAP Crossover Setup
Running Analysis - VWAP Retest Setup
2022-02-02 09:30:00 - Running Analysis on Ticker Symbol:  AAPL
[*********************100%***********************]  1 of 1 completed
Running Analysis - EMA9 + VWAP Crossover Setup
Running Analysis - VWAP Retest Setup
2022-02-02 09:30:00 - Running Analysis on Ticker Symbol:  TSLA
[*********************100%***********************]  1 of 1 completed
Running Analysis - EMA9 + VWAP Crossover Setup
Running Analysis - VWAP Retest Setup
2022-02-02 09:33:00 - Running Analysis on Ticker Symbol:  MSFT
[*********************100%***********************]  1 of 1 completed
Running Analysis - EMA9 + VWAP Crossover Setup
Running Analysis - VWAP Retest Setup
2022-02-02 09:33:00 - Running Analysis on Ticker Symbol:  AAPL
[*********************100%***********************]  1 of 1 completed
Runn

<ipython-input-5-39ca2561f04a>:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-39ca2561f04a>:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-39ca2561f04a>:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\

...

<ipython-input-5-39ca2561f04a>:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-39ca2561f04a>:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-39ca2561f04a>:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-39ca2561

[*********************100%***********************]  1 of 1 completed
Running Analysis - TripleEMA Setup
Running Analysis - EMA9 + VWAP Crossover Setup
Running Analysis - VWAP Retest Setup
2022-02-02 15:24:00 - Running Analysis on Ticker Symbol:  TSLA


<ipython-input-5-39ca2561f04a>:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-39ca2561f04a>:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-39ca2561f04a>:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\

[*********************100%***********************]  1 of 1 completed
Running Analysis - TripleEMA Setup
Running Analysis - EMA9 + VWAP Crossover Setup
Running Analysis - VWAP Retest Setup
2022-02-02 15:27:00 - Running Analysis on Ticker Symbol:  MSFT
[*********************100%***********************]  1 of 1 completed
Running Analysis - TripleEMA Setup
Running Analysis - EMA9 + VWAP Crossover Setup
Running Analysis - VWAP Retest Setup
2022-02-02 15:27:00 - Running Analysis on Ticker Symbol:  AAPL


<ipython-input-5-39ca2561f04a>:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-39ca2561f04a>:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-39ca2561f04a>:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-39ca2561

In [ ]:
result_df.loc[result_df['crossover_position'] == 1]

In [ ]:
result_df.loc[result_df['crossover_position'] == -1]

In [ ]:
result_df.loc[result_df['triple_ema_hold_rule_one'] == -1]

In [ ]:
result_df.loc[result_df['triple_ema_retest_rule_two'] == -1]

In [ ]:
# Pull Live Data
intraday_data_new = yf.download(tickers='TWTR', period='2d', interval='1m', auto_adjust=True) 
intraday_data_new.tail()

In [ ]:
test_intraday_data = yf.download(tickers='MSFT', period='1d', interval='1m', auto_adjust=True)
test_intraday_data.head()
datetime_time = datetime.datetime.combine(today_date, datetime.time(9, 30, 0))
test_intraday_data.loc[:datetime_time].head(10)

In [ ]:
from statistics import mean
from matplotlib import pyplot  as plt
%matplotlib

mean_time_taken = round(mean(time_list),4)

fig = plt.figure()
plt.plot(time_list,'g', label='All 3 Methods')
plt.plot(triple_ema_time_list,'b', label='Triple EMA')
plt.plot(ema9_crossover_time_list,'k', label='EMA9 + VWAP Combo')
plt.plot(vwap_time_list,'r', label='VWAP Retest')
plt.axhline(y = mean_time_taken, color = 'm', linestyle = '--')
plt.xlabel('Iterations')
plt.ylabel('Time Taken (seconds)')
plt.title('Processing time taken for 1 ticker across each iteration')
plt.grid(True)
plt.legend(["All 3 Methods", "Triple EMA", "EMA9 + VWAP Combo", "VWAP Retest", 'Mean Time Taken'], loc="upper left")
plt.show()

from statistics import mean
round(mean(time_list),4)

In [ ]:
print(watchlist_dict['MSFT'])

In [ ]:
print(watchlist_dict)